# RR Model Pipeline Template

Lưu đồ thực thi: nạp dữ liệu → chuẩn hóa schema → build transition/PD → (EAD dòng tiền) → ECL.

Điều chỉnh `config.py` để chỉ định nguồn dữ liệu (parquet/oracle/excel) và cột EAD cho PD vs ECL.

In [ ]:
from src.data_loader import load_data
from src.config import CFG

df = load_data()
df.shape, df.head()

## 1) Transition + PD forward

- Build ma trận Markov theo PRODUCT_TYPE/RISK_SCORE/MOB dùng `CFG['ead_pd']`.
- Cache PD forward per key (state, mob, product, score).

In [ ]:
from src.rollrate.transition import compute_transition, make_pairs
from src.rollrate.pd_forward import compute_forward_pd_one_record
from src.config import BUCKETS_CANON

# Ví dụ: build ma trận global (demo). Thực tế nên loop theo PRODUCT_TYPE/RISK_SCORE.
pairs = make_pairs(df)
P = compute_transition(df, value_col=CFG['ead_pd'])
P

## 2) EAD (dòng tiền) cho ECL

Sẽ thay bằng builder EAD_ECL/profiles sau khi thống nhất công thức.
Tạm thời dùng `CFG['ead_ecl']` để chạy ECL với EAD hiện có.

In [ ]:
from src.rollrate.ECL_final import compute_ecl_final

# placeholders: matrices_by_mob, parent_fallback cần được build trước (transition pipeline).
# df_sched: schedule gốc nếu có; để None nếu chưa chuẩn bị.

matrices_by_mob = {}
parent_fallback = None
df_sched = None

# df_current nên là snapshot mới nhất (lọc ngoài).
df_current = df.copy()

df_ecl = compute_ecl_final(
    df_current=df_current,
    df_sched=df_sched,
    matrices_by_mob=matrices_by_mob,
    parent_fallback=parent_fallback,
)
df_ecl.head()

## 3) Forecast + Reporting

- Dùng `forecast_report` để dự báo EAD theo state.
- Xuất Excel/CSV tùy nhu cầu.